This is Ruchi's original attmpt at obtaining music data from various sources: LastFM, Genius Lyrics, and Musicbrainz. It shows us what kind of returns we can obtain from each site. 

In [ ]:
# Load libraries
import os
from dotenv import load_dotenv
import musicbrainzngs
import pprint
import requests
import lyricsgenius

In [ ]:
# Set a user agent for MusicBrainz
#musicbrainzngs.set_useragent("music-trend-project", "0.1", "your_email@example.com")
musicbrainzngs.set_useragent("music-study", "0.1", "ruchipatil@outlook.com")

# Search for a track
result = musicbrainzngs.search_recordings(query="Euphoria", limit=1)
track = result['recording-list'][0]

print("title: ", track['title'])
print("artist: ", track['artist-credit'][0]['name'])
print("MBID: ", track['id'])
mbid = track['id']
# for k, v in track.items():
#     print("--------------------------------------------------------")
#     print(f'{k}: {v}')

pprint.pprint(track)

In [ ]:
# Use mbid from previous step
mbid = track['id']  
url = f"https://acousticbrainz.org/api/v1/{mbid}/high-level"

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print("🎛️ Acoustic Features (high-level):")
    print(data["highlevel"])
else:
    print(f"❌ Could not fetch features for {mbid}")

In [ ]:
# mbid = 'eb1ce8de-c51a-4e6a-a03f-15db3163f67e'

url = f"https://acousticbrainz.org/api/v1/{mbid}/high-level"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("features according to acoustic brainz")
    pprint.pprint(data)
else:
    print("error", response.status_code)

The unique mbid looks like a great way to pull music information from Musicbrainz. We need to set up our API call requests to pull all songs for a given set of years. We should have JSON returns, as these are easy to dump into a Pandas Dataframe and then we can transform into CSV for saving if needed. 
We then need to run all songs through another API call set for Genius Lyrics to get the lyrics. (into separate file, then merge?)
Once we have all the songs and lyrics, we need to select the subset that has English lyrics.

In [ ]:
# Set user agent for LastFM
load_dotenv()

lastfm_key = os.getenv("LAST_FM_API_KEY")
artist = "Kendrick Lamar"
track_name = "Euphoria"

url = f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&artist={artist}&track={track_name}&api_key={lastfm_key}&format=json"
response = requests.get(url)
if response.status_code == 200:
    info = response.json()
    print("🎧 Playcount:", info['track']['playcount'])
    print("🏷️ Tags:", [tag['name'] for tag in info['track']['toptags']['tag']])
else:
    print("❌ Last.fm request failed")

In [ ]:
# Second attempt LastFM
load_dotenv()

lastfm_key = os.getenv("LAST_FM_API_KEY")
artist = "britney"
track = "womanizer"

url = f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&artist={artist}&track={track}&api_key={lastfm_key}&format=json"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    pprint.pprint(data)
else:
    print("erm idk error: ", response.status_code)

In [ ]:
# Set user agent for Genius Lyrics
load_dotenv()
genius_token = os.getenv("GENIUS_API_TOKEN")
genius = lyricsgenius.Genius(genius_token)

song = genius.search_song("womanizer", "britney")

if song:
    print("yay we found song: ", song.title, "by", song.artist)
    print(song.lyrics[:500])                                    # show first 500 chars
else:                                                           # need to set 'continue' so function doesn't stop
    print("WOMPWOMP song not found")

In [ ]:
import requests
DISCOGS_TOKEN = "xmWNqVMtvKoFllImiTkEbgIMpgmdVRVggXqrWJPQ"   #"your_discogs_token_here"

def search_discogs_by_year(year, per_page=10):
    url = "https://api.discogs.com/database/search"
    headers = {"User-Agent": "YourAppName/1.0"}
    params = {
        "token": DISCOGS_TOKEN,
        "year": year,
        "type": "release",
        "format": "Album",
        "per_page": per_page,
        "page": 1
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    results = data.get("results", [])
    return results

results = search_discogs_by_year(2020, per_page=20)
#print(results[0])
import pprint
pprint.pprint(results[:3])

In [ ]:
# Can we use Ruchi's Genius Lyrics setup?
import pprint
import requests
import lyricsgenius

GENIUS_API_TOKEN = "r6vW9hyUck05RRuvBF4L1f5MjirVAeuHO0tauARd-aB1pt7xscCUVzMnFoFC2WTS"
genius = lyricsgenius.Genius(GENIUS_API_TOKEN)

song = genius.search_song("womanizer", "britney")

if song:
    pos = int(song.lyrics.find('[Verse 1]'))
    print(song.lyrics[pos:])                                  # show first 500 chars print(song.lyrics[:500])
else:                                                           # need to change index to only get song lyrics
    print("Song not found")

# Genius has option to strip the headers too...

In [ ]:
import lyricsgenius

GENIUS_API_TOKEN = "r6vW9hyUck05RRuvBF4L1f5MjirVAeuHO0tauARd-aB1pt7xscCUVzMnFoFC2WTS"

# Initialize the Genius API client
genius = lyricsgenius.Genius(GENIUS_API_TOKEN)

# Other Genius settings for cleaner results
genius.verbose = False
genius.remove_section_headers = True
genius.skip_non_songs = True

song = genius.search_song("womanizer", "britney")

if song:
    print(f"🎵 Found song: {song.title} by {song.artist}")
    print(song.lyrics)  # Full lyrics to see results
else:
    print("🚫 Song not found.")